<a href="https://colab.research.google.com/github/arihantbansal/SAiDL-Spring-Assignment-2022/blob/main/NLP/trec_mixup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Mixup on TREC Dataset

## Import libraries

In [28]:
!pip install datasets
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4 MB 1.3 MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-py3-none-any.whl size=98051301 sha256=d77ab0fee3ca8011312d4a9ce75d1602d9ac9fc47d63d31b6b524f78426c7301
  Stored in directory: /tmp/pip-ephem-wheel-cache-_ivujo_m/wheels/69/c5/b8/4f1c029d89238734311b3269762ab2ee325a42da2ce8edb997
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

# Utility imports
import spacy
import re
import string
import time

# Extras
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

from datasets import load_dataset # using 🤗 HugggingFace datasets library


## Set Random Seed

In [4]:
SEED = 420

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## Load dataset

In [5]:
dataset = load_dataset("trec")

Downloading:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Using custom data configuration default


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/336k [00:00<?, ?B/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset trec downloaded and prepared to /root/.cache/huggingface/datasets/trec/default/1.1.0/751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

## Exploring the dataset

In [6]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['label-coarse', 'label-fine', 'text'],
        num_rows: 5452
    })
    test: Dataset({
        features: ['label-coarse', 'label-fine', 'text'],
        num_rows: 500
    })
})


In [7]:
dataset.keys()

dict_keys(['train', 'test'])

In [8]:
print(dataset["train"].dataset_size)
print(dataset["train"].description)
print(dataset["train"].features)


413073
The Text REtrieval Conference (TREC) Question Classification dataset contains 5500 labeled questions in training set and another 500 for test set. The dataset has 6 labels, 47 level-2 labels. Average length of each sentence is 10, vocabulary size of 8700.

Data are collected from four sources: 4,500 English questions published by USC (Hovy et al., 2001), about 500 manually constructed questions for a few rare classes, 894 TREC 8 and TREC 9 questions, and also 500 questions from TREC 10 which serves as the test set.

{'label-coarse': ClassLabel(num_classes=6, names=['DESC', 'ENTY', 'ABBR', 'HUM', 'NUM', 'LOC'], names_file=None, id=None), 'label-fine': ClassLabel(num_classes=47, names=['manner', 'cremat', 'animal', 'exp', 'ind', 'gr', 'title', 'def', 'date', 'reason', 'event', 'state', 'desc', 'count', 'other', 'letter', 'religion', 'food', 'country', 'color', 'termeq', 'city', 'body', 'dismed', 'mount', 'money', 'product', 'period', 'substance', 'sport', 'plant', 'techmeth', 'vol

In [9]:
# Sample data point in TREC

dataset["train"][0]

{'label-coarse': 0,
 'label-fine': 0,
 'text': 'How did serfdom develop in and then leave Russia ?'}

## Tokenize Data

In [29]:
nlp = spacy.load("en_core_web_md")

def tokenize(text):
	text = re.sub(r"[^\x00-\x7F]+", " ", str(text))
	regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]') # remove punctuation and numbers
	no_punctuation = regex.sub(" ", text.lower())
	return [token.text for token in nlp.tokenizer(no_punctuation)]


OSError: ignored

In [11]:
train_ds = pd.DataFrame(dataset["train"])
train_ds.head()

,label-coarse,label-fine,text
0,0,0,How did serfdom develop in and then leave Russ...
1,1,1,What films featured the character Popeye Doyle ?
2,0,0,How can I find a list of celebrities ' real na...
3,1,2,What fowl grabs the spotlight after the Chines...
4,2,3,What is the full form of .com ?


In [12]:
test_ds = pd.DataFrame(dataset["test"])
test_ds.head()

,label-coarse,label-fine,text
0,4,40,How far is it from Denver to Aspen ?
1,5,21,"What county is Modesto , California in ?"
2,3,12,Who was Galileo ?
3,0,7,What is an atom ?
4,4,8,When did Hawaii become a state ?


In [14]:
train_ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5452 entries, 0 to 5451
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   label-coarse  5452 non-null   int64 
 1   label-fine    5452 non-null   int64 
 2   text          5452 non-null   object
dtypes: int64(2), object(1)
memory usage: 127.9+ KB


In [24]:
train_ds["text"] = train_ds["text"].str.strip()

In [25]:
counts = Counter()
for index, row in train_ds.iterrows():
  counts.update(tokenize(train_ds["text"]))

In [26]:
len(counts.keys())

65

In [27]:
counts

Counter({' ': 32712,
         '  ': 10904,
         '       ': 5452,
         '        ': 5452,
         '          ': 5452,
         '           ': 5452,
         '            ': 5452,
         '                   ': 5452,
         '                      ': 5452,
         '                       ': 5452,
         '                            ': 5452,
         '                             ': 5452,
         '                                                                              ': 5452,
         'a': 10904,
         'after': 5452,
         'and': 5452,
         'australia': 5452,
         'camel': 5452,
         'can': 5452,
         'celebrities': 5452,
         'character': 5452,
         'china': 5452,
         'chines': 5452,
         'com': 5452,
         'cooking': 5452,
         'currency': 10904,
         'develop': 5452,
         'did': 5452,
         'doyle': 5452,
         'dtype': 5452,
         'featured': 5452,
         'films': 5452,
         'find': 5452,
       

In [ ]:
# from transformers import BertTokenizer
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

100%|██████████| 231508/231508 [00:01<00:00, 129681.82B/s]


In [ ]:
dataset.set_format(type="torch", columns=["text", "label-coarse"])
train_dataloader = DataLoader(dataset["train"], batch_size=256)
next(iter(train_dataloader))